In [ ]:
# 지금까지 DB에 데이터를 쌓는 것을 한 것이고
# 이번엔 쌓아놓은 DB 내 데이터를 조회하는 API를 만들어보자

# 종목코드 대신 기업명으로 조회하는 기능도 추가될 것이며
# 조회일자 형식을 다르게 입력하더라도 자동으로 양식을 보정하게 할 것이다

# 클래스 구조는 다음과 같다
'''
class MarketDB:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
    
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
    
    def get_comp_info(self):
        """COMPANY_INFO 테이블에서 읽어와서 codes에 저장"""
    
    def get_daily_price(self, code, start_date=None, end_date=None):
        """KRX 종목별 시세를 DataFrame형태로 반환"""
'''

class MarketDB:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
    
    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
    
    def get_comp_info(self):
        """COMPANY_INFO 테이블에서 읽어와서 codes에 저장"""
    
    def get_daily_price(self, code, start_date=None, end_date=None):
        """KRX 종목별 시세를 DataFrame형태로 반환"""